In [11]:
import sys
!"{sys.executable}" -m pip install requests pandas kafka-python

In [13]:
import time
import requests
import json
import pandas as pd
from kafka import KafkaProducer

API_URL      = "https://danepubliczne.imgw.pl/api/data/hydro2/"
KAFKA_BROKER = "localhost:9092"
TOPIC        = "hydro2"
POLL_INTERVAL = 60

In [15]:
def fetch_hydro_data(url: str):
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.json()

data = fetch_hydro_data(API_URL)
df = pd.DataFrame(data)
df.head()

,kod_stacji,nazwa_stacji,lon,lat,stan,stan_data,przelyw,przeplyw_data
0,150160330,SZCZYTNA,16.443056,50.415556,133,2025-05-05 19:20:00,None,None
1,150160340,SARNY,16.465833,50.547778,74,2025-05-05 19:20:00,None,None
2,150160350,SZALEJÓW GÓRNY,16.537222,50.418333,118,2025-05-05 19:20:00,None,None
3,150160360,STARKÓW,16.58,50.3775,154,2025-05-05 19:20:00,None,None
4,150160370,TOPOLICE,16.609167,50.366944,140,2025-05-05 19:20:00,None,None


In [17]:
producer = KafkaProducer(
    bootstrap_servers=[KAFKA_BROKER],
    value_serializer=lambda v: json.dumps(v).encode("utf-8"),
    linger_ms=10
)

In [ ]:
try:
    while True:
        records = fetch_hydro_data(API_URL)
        for rec in records:
            producer.send(TOPIC, rec)
        producer.flush()
        print(f"{len(records)} rekordów wysłano.")
        time.sleep(POLL_INTERVAL)
except KeyboardInterrupt:
    print("Pętlę przerwano.")
finally:
    producer.close()
    print("Producer zamknięty.")


859 rekordów wysłano.
859 rekordów wysłano.
859 rekordów wysłano.
859 rekordów wysłano.
859 rekordów wysłano.
859 rekordów wysłano.
859 rekordów wysłano.
859 rekordów wysłano.
859 rekordów wysłano.
859 rekordów wysłano.
859 rekordów wysłano.
859 rekordów wysłano.
859 rekordów wysłano.
859 rekordów wysłano.
859 rekordów wysłano.
